In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, auc
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Input, Multiply, TimeDistributed, Softmax, Lambda, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from xgboost import XGBClassifier
from tensorflow.keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
        

def print_results(outcome_variable, lr_prob):
        ### Training data:
        # Load data from a CSV file
        df = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-04-18/df_dl_continous_lr.csv')
        df_mask = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-04-18/df_dl_continous_mask.csv')
        df_delta = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-04-18/df_dl_continous_delta.csv')

        Africa=["Zambia", "Kenya", "Ghana"]
        Asia=["Pakistan", "India-CMC", "India-SAS"]
    
       # Drop rows with NaN in the target column before any operations
        df = df.dropna(subset=[outcome_variable])
        df = df[df['SITE'].isin(Asia)]
        df = df.reset_index(drop=True)
    
        df_mask = df_mask.dropna(subset=[outcome_variable])
        df_mask = df_mask[df_mask['SITE'].isin(Asia)]
        df_mask = df_mask.reset_index(drop=True)
    
        df_delta = df_delta.dropna(subset=[outcome_variable])
        df_delta = df_delta[df_delta['SITE'].isin(Asia)]
        df_delta = df_delta.reset_index(drop=True)
        
        features_categorical = ["TYPE_VISIT",
                    "INF_PRES_CEPH","INF_PRES_BREECH","INF_PRES_TRANS","INF_PRES_BROW","INF_PRES_OTHER",
                    "STILLBIRTH_IND","PRETERM_IND","CESARIAN_IND","NUM_MISCARRIAGE_ind",
                    "DEPR_EVER",
                    "PAID_WORK","GPARITY_2","GPARITY_1","WEALTH_QUINT_1","WEALTH_QUINT_2","WEALTH_QUINT_3","WEALTH_QUINT_4","SCHOOL_MORE10","WATER_IMPROVED","TOILET_IMPROVED","STOVE_FUEL","HH_SMOKE","SMOKE","CHEW_TOBACCO","CHEW_BETELNUT",
                    "BMI_LEVEL_ENROLL_underweight","BMI_LEVEL_ENROLL_overweight","BMI_LEVEL_ENROLL_obese",
                    "GWG_ADEQUACY_inadequate","GWG_ADEQUACY_excessive",
                    "MEM_CES","MEM_ART","MEM_SPON","LABOR_ANY","PRO_LABOR","OBS_LABOR","MAT_CES_ANY","BIRTH_FACILITY",
                    "HTN","DIAB_OVERT_ANY","HIV_POSITIVE_ENROLL","TB_SYMP_POSITIVE_ENROLL","MAL_POSITIVE_ENROLL","STI_POSITIVE_ENROLL",
                    "GES_HTN","DIAB_GEST_ANY","HIV_POSITIVE_ANY_ANC","TB_SYMP_POSITIVE_ANY_ANC","MAL_POSITIVE_ANY_ANC","HBV_POSITIVE_ANY_ANC","HCV_POSITIVE_ANY_ANC","STI_POSITIVE_ANY_ANC",
                    "INF_SEX","INF_HYPERBILI_NICE","BREASTFED",
                    "NUM_FETUS_2","NUM_FETUS_3","INF_ANOMALY","PREVIA",
                    "INF_PSBI_IPC"]
        
        features_continous = ["AGE_GROUP","MAT_AGE","MUAC_ENROLL","BMI_ENROLL","NUM_FETUS","NUM_MISCARRIAGE","GESTAGEBIRTH_ANY","BWEIGHT_ANY","CROWDING_IND",
                              "M01_AFI_PERES",
                              "M08_IRON_TOT_UGDL_LBORRES", "M08_FERRITIN_LBORRES","M08_IRON_HEP_LBORRES"]
        
        features_static = ["INF_PRES_CEPH","INF_PRES_BREECH","INF_PRES_TRANS","INF_PRES_BROW","INF_PRES_OTHER",
                    "STILLBIRTH_IND","PRETERM_IND","CESARIAN_IND","NUM_MISCARRIAGE_ind",
                    "DEPR_EVER",
                    "PAID_WORK","GPARITY_2","GPARITY_1","WEALTH_QUINT_1","WEALTH_QUINT_2","WEALTH_QUINT_3","WEALTH_QUINT_4","SCHOOL_MORE10","WATER_IMPROVED","TOILET_IMPROVED","STOVE_FUEL","HH_SMOKE","SMOKE","CHEW_TOBACCO","CHEW_BETELNUT","CROWDING_IND",
                    "BMI_LEVEL_ENROLL_underweight","BMI_LEVEL_ENROLL_overweight","BMI_LEVEL_ENROLL_obese",
                    "GWG_ADEQUACY_inadequate","GWG_ADEQUACY_excessive",
                    "MEM_CES","MEM_ART","MEM_SPON","LABOR_ANY","PRO_LABOR","OBS_LABOR","MAT_CES_ANY","BIRTH_FACILITY",
                    "HTN","DIAB_OVERT_ANY","HIV_POSITIVE_ENROLL","TB_SYMP_POSITIVE_ENROLL","MAL_POSITIVE_ENROLL","STI_POSITIVE_ENROLL",
                    "GES_HTN","DIAB_GEST_ANY","HIV_POSITIVE_ANY_ANC","TB_SYMP_POSITIVE_ANY_ANC","MAL_POSITIVE_ANY_ANC","HBV_POSITIVE_ANY_ANC","HCV_POSITIVE_ANY_ANC","STI_POSITIVE_ANY_ANC",
                    "INF_SEX","INF_HYPERBILI_NICE","BREASTFED","GESTAGEBIRTH_ANY","BWEIGHT_ANY",
                    "INF_ANOMALY",
                    "INF_PSBI_IPC",
                    "NUM_FETUS_2","NUM_FETUS_3","MAT_AGE","MUAC_ENROLL",
                    "AGE_GROUP","BMI_ENROLL","NUM_FETUS","NUM_MISCARRIAGE"]
        
        features_temporal = ["TYPE_VISIT",
                             "M01_AFI_PERES",
                             "PREVIA",
                             "M08_IRON_TOT_UGDL_LBORRES","M08_FERRITIN_LBORRES","M08_IRON_HEP_LBORRES"]

        features_to_remove_by_outcome = {
        "STILLBIRTH_SIGNS_LIFE": ["INF_SEX","INF_HYPERBILI_NICE","BREASTFED","BWEIGHT_ANY","INF_PSBI_IPC"],
        "PRETERMBIRTH_LT37": ["INF_HYPERBILI_NICE","BREASTFED","GESTAGEBIRTH_ANY","BWEIGHT_ANY","INF_PSBI_IPC","LABOR_ANY","PRO_LABOR","OBS_LABOR","MAT_CES_ANY","BIRTH_FACILITY"],
        "LBW2500_ANY": ["INF_HYPERBILI_NICE","BREASTFED","BWEIGHT_ANY","INF_PSBI_IPC","LABOR_ANY","PRO_LABOR","OBS_LABOR","MAT_CES_ANY","BIRTH_FACILITY"],
        "INF_PSBI_IPC": ["INF_HYPERBILI_NICE","INF_PSBI_IPC"],
        "SVN": ["INF_SEX","INF_HYPERBILI_NICE","GESTAGEBIRTH_ANY","BWEIGHT_ANY","INF_PSBI_IPC","BREASTFED","LABOR_ANY","PRO_LABOR","OBS_LABOR","MAT_CES_ANY","BIRTH_FACILITY"],
        "NEARMISS": ["INF_SEX","INF_HYPERBILI_NICE","BREASTFED","GESTAGEBIRTH_ANY","BWEIGHT_ANY","INF_PSBI_IPC"]
        }
    
        features_to_remove = features_to_remove_by_outcome.get(outcome_variable, [])
        
        # Remove features from the lists
        features_categorical = [f for f in features_categorical if f not in features_to_remove]
        features_continous = [f for f in features_continous if f not in features_to_remove]
        features_static = [f for f in features_static if f not in features_to_remove]
        features_temporal = [f for f in features_temporal if f not in features_to_remove]
        
        # Separate categorical and continuous features
        df_categorical = df[features_categorical]
        df_continuous = df[features_continous]
        
        # Rescale continuous features to (0, 1) range
        scaler = MinMaxScaler()
        df_continuous_scaled = pd.DataFrame(scaler.fit_transform(df_continuous), columns=features_continous)
        
        # Concatenate scaled continuous features with categorical features
        X = pd.concat([df_categorical, df_continuous_scaled], axis=1)
        
        ##Static and temporal data
        X_static = X[features_static]
        
        X_temporal = X[features_temporal]
        X_temporal_delta = df_delta[features_temporal]
        X_temporal_mask = df_mask[features_temporal]
        
        y = df[outcome_variable]
        
        # One-hot encoding for non-binary categorical features in X
        X = pd.get_dummies(X, columns=['TYPE_VISIT'], dummy_na=False)
        X_temporal = pd.get_dummies(X_temporal, columns=['TYPE_VISIT'], dummy_na=False)
        X_temporal_delta = pd.get_dummies(X_temporal_delta, columns=['TYPE_VISIT'], dummy_na=False)
        X_temporal_mask = pd.get_dummies(X_temporal_mask, columns=['TYPE_VISIT'], dummy_na=False)
        
        # Preprocess for Keras models (3D matrix)
        n_samples = X_temporal.shape[0] // 5 
        X_static_keras = X_static.iloc[::5].values.astype('float32')
        
        X_temporal_keras = X_temporal.values.reshape(n_samples, 5, X_temporal.shape[1]).astype('float32')
        X_temporal_delta_keras = X_temporal_delta.values.reshape(n_samples, 5, X_temporal_delta.shape[1]).astype('float32')
        X_temporal_mask_keras = X_temporal_mask.values.reshape(n_samples, 5, X_temporal_mask.shape[1]).astype('float32')
        
        X_keras = X.values.reshape(n_samples, 5, X.shape[1]).astype('float32')
        
        # Align y for Keras models
        y_keras = y.iloc[::5].values.astype('float32')

        ### Test data:
        # Load data from a CSV file
        df_test = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-04-18/df_dl_continous_lr.csv')
        df_test_mask = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-04-18/df_dl_continous_mask.csv')
        df_test_delta = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-04-18/df_dl_continous_delta.csv')
        
       # Drop rows with NaN in the target column before any operations
        df_test = df_test.dropna(subset=[outcome_variable])
        df_test = df_test[df_test['SITE'].isin(Africa)]
        df_test = df_test.reset_index(drop=True)
    
        df_test_mask = df_test_mask.dropna(subset=[outcome_variable])
        df_test_mask = df_test_mask[df_test_mask['SITE'].isin(Africa)]
        df_test_mask = df_test_mask.reset_index(drop=True)
    
        df_test_delta = df_test_delta.dropna(subset=[outcome_variable])
        df_test_delta = df_test_delta[df_test_delta['SITE'].isin(Africa)]
        df_test_delta = df_test_delta.reset_index(drop=True)
    
        # Separate categorical and continuous features
        df_test_categorical = df_test[features_categorical]
        df_test_continuous = df_test[features_continous]
        
        # Rescale continuous features to (0, 1) range
        df_test_continuous_scaled = pd.DataFrame(scaler.fit_transform(df_test_continuous), columns=features_continous)
        
        # Concatenate scaled continuous features with categorical features
        X_test = pd.concat([df_test_categorical, df_test_continuous_scaled], axis=1)
          
        ##Static and temporal data
        X_test_static = X_test[features_static]
        
        X_test_temporal = X_test[features_temporal]
        X_test_temporal_delta = df_test_delta[features_temporal]
        X_test_temporal_mask = df_test_mask[features_temporal]

        y_test_soft = df_test[lr_prob]
        y_test = df_test[outcome_variable]
        threshold = (df_test[outcome_variable] == 1).mean()
        
        # One-hot encoding for non-binary categorical features in X
        X_test = pd.get_dummies(X_test, columns=['TYPE_VISIT'], dummy_na=False)
        X_test_temporal = pd.get_dummies(X_test_temporal, columns=['TYPE_VISIT'], dummy_na=False)
        X_test_temporal_delta = pd.get_dummies(X_test_temporal_delta, columns=['TYPE_VISIT'], dummy_na=False)
        X_test_temporal_mask = pd.get_dummies(X_test_temporal_mask, columns=['TYPE_VISIT'], dummy_na=False)
        
        # Preprocess for Keras models (3D matrix)
        n_test_samples = X_test_temporal.shape[0] // 5 
        X_test_static_keras = X_test_static.iloc[::5].values.astype('float32')
        
        X_test_temporal_keras = X_test_temporal.values.reshape(n_test_samples, 5, X_test_temporal.shape[1]).astype('float32')
        X_test_temporal_delta_keras = X_test_temporal_delta.values.reshape(n_test_samples, 5, X_test_temporal_delta.shape[1]).astype('float32')
        X_test_temporal_mask_keras = X_test_temporal_mask.values.reshape(n_test_samples, 5, X_test_temporal_mask.shape[1]).astype('float32')
        
        X_test_keras = X_test.values.reshape(n_test_samples, 5, X_test.shape[1]).astype('float32')
        
        # Align y for Keras models
        y_test_soft_keras = y_test_soft.iloc[::5].values.astype('float32')
        y_test_keras = y_test.iloc[::5].values.astype('float32')

        #### GRUD With Static Attention
        class GRUDWithStaticAttention(nn.Module):
            def __init__(self, input_size, hidden_size, output_size, mean_values, static_input_size, static_embedding_size, dropout_rate=0.5):
                """
                GRU-D model with attention mechanism generated from static data.
                
                Args:
                    input_size (int): Number of input features for temporal data.
                    hidden_size (int): Number of hidden units in GRU.
                    output_size (int): Number of output classes (for classification tasks).
                    mean_values (torch.Tensor): Empirical mean values for each input feature.
                    static_input_size (int): Number of input features for static data.
                    static_embedding_size (int): Size of the embedding for static data.
                """
                super(GRUDWithStaticAttention, self).__init__()
                
                # Temporal (GRU-D) parameters
                self.input_size = input_size
                self.hidden_size = hidden_size
                self.output_size = output_size
                
                self.mean_values = mean_values
                
                self.gamma_x = nn.Linear(input_size, input_size)
                
                # GRU gate parameters (temporal part) with combined input, hidden state, and mask
                self.zl = nn.Linear(input_size + hidden_size + input_size, hidden_size)  # Update gate
                self.rl = nn.Linear(input_size + hidden_size + input_size, hidden_size)  # Reset gate
                self.hl = nn.Linear(input_size + hidden_size + input_size, hidden_size)  # Candidate hidden state
        
                # Embedding or processing static data
                self.static_fc = nn.Linear(static_input_size, static_embedding_size)
        
                # Attention mechanism based on static data
                self.attention = nn.Linear(static_embedding_size, hidden_size)
        
                # Fully connected output layer
                self.dropout = nn.Dropout(dropout_rate)
                self.concat_fc = nn.Linear(hidden_size + static_embedding_size, output_size)

        
            def forward(self, x, m, delta, static_data):
                """
                Forward pass of GRU-D with static data attention.
                
                Args:
                    x (torch.Tensor): Temporal input data [batch_size, seq_len, input_size].
                    m (torch.Tensor): Masking vector [batch_size, seq_len, input_size].
                    delta (torch.Tensor): Time intervals [batch_size, seq_len, input_size].
                    static_data (torch.Tensor): Static data input [batch_size, static_input_size].
                """
                batch_size, seq_len, _ = x.size()
                
                # Process static data to generate attention weights
                static_embed = torch.relu(self.static_fc(static_data))  # [batch_size, static_embedding_size]
                
                # Generate attention weights from static data
                attention_weights = torch.sigmoid(self.attention(static_embed))  # [batch_size, hidden_size]
                
                # Initialize hidden state
                h = torch.zeros(batch_size, self.hidden_size).to(x.device)
                
                for t in range(seq_len):
                    # Corrected input decay mechanism
                    gamma_x_t = torch.exp(-F.relu(self.gamma_x(delta[:, t, :])))
                    x_t_hat = m[:, t, :] * x[:, t, :] + (1 - m[:, t, :]) * (gamma_x_t * self.mean_values + (1 - gamma_x_t) * x[:, t, :])
                    
                    # Concatenate input, hidden state, and mask
                    combined = torch.cat([x_t_hat, h, m[:, t, :]], dim=-1)
        
                    # GRU gates
                    r_t = torch.sigmoid(self.rl(combined))  # Reset gate
                    z_t = torch.sigmoid(self.zl(combined))  # Update gate
                    h_tilde = torch.tanh(self.hl(torch.cat([x_t_hat, r_t * h, m[:, t, :]], dim=-1)))  # Candidate hidden state
        
                    # Update hidden state
                    h = (1 - z_t) * h + z_t * h_tilde
        
                # Apply attention weights to the final hidden state
                h_weighted = attention_weights * h  # [batch_size, hidden_size]

                h_concat = self.dropout(torch.cat([h_weighted, static_embed], dim=-1))  # [B, hidden + static_embedding_size]
            
                output = torch.sigmoid(self.concat_fc(h_concat)) 
                return output
        
        # Training Parameters
        input_size = X_temporal_keras.shape[2]  # Temporal input features
        hidden_size = 32  # GRU hidden state size
        output_size = 1  # Binary classification
        static_input_size = X_static_keras.shape[1]  # Static input features
        static_embedding_size = 32  # Embedding size for static features
        mean_values = torch.tensor(np.nanmean(X_temporal_keras, axis=(0, 1)), dtype=torch.float32)  # Temporal feature means

        # Convert data to PyTorch tensors
        x = torch.tensor(X_temporal_keras, dtype=torch.float32)  # Temporal input
        mask = torch.tensor(X_temporal_mask_keras, dtype=torch.float32)  # Masking vector
        delta = torch.tensor(X_temporal_delta_keras, dtype=torch.float32)  # Time intervals
        static_data = torch.tensor(X_static_keras, dtype=torch.float32)  # Static input
        y = torch.tensor(y_keras, dtype=torch.float32)  # Binary labels
        
        def GRUD_Static_Model_trained(model_class, x, mask, delta, static_data, y, n_epochs=100, learning_rate=0.01):
            model = model_class(
                input_size=x.shape[2],
                hidden_size=hidden_size,
                output_size=output_size,
                mean_values=mean_values,
                static_input_size=static_input_size,
                static_embedding_size=static_embedding_size,
            )
        
            criterion = nn.BCELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
            for epoch in range(n_epochs):
                model.train()
                optimizer.zero_grad()
                outputs = model(x, mask, delta, static_data)
                loss = criterion(outputs.squeeze(), y.squeeze())
                loss.backward()
                optimizer.step()
        
            model.eval()
            return model

        trained_model = GRUD_Static_Model_trained(
            GRUDWithStaticAttention,
            x,
            mask,
            delta,
            static_data,
            y,
            n_epochs=100,
            learning_rate=0.01
        )
        
        # Convert test data to torch tensors
        x_test = torch.tensor(X_test_temporal_keras, dtype=torch.float32)
        mask_test = torch.tensor(X_test_temporal_mask_keras, dtype=torch.float32)
        delta_test = torch.tensor(X_test_temporal_delta_keras, dtype=torch.float32)
        static_test = torch.tensor(X_test_static_keras, dtype=torch.float32)
        y_test_soft = torch.tensor(y_test_soft_keras, dtype=torch.float32)
        y_test_true = y_test_keras.astype(int)  # Convert to 0/1 labels

        def fine_tune_model(model, x, mask, delta, static_data, y_soft, n_epochs=100, learning_rate=0.01):
            model.train()
            criterion = nn.BCELoss()  # Accepts probabilistic labels
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
            for epoch in range(n_epochs):
                optimizer.zero_grad()
                outputs = model(x, mask, delta, static_data).squeeze()
                loss = criterion(outputs, y_soft)
                loss.backward()
                optimizer.step()
            
            model.eval()
            return model

        fine_tuned_model = fine_tune_model(
            trained_model,
            x_test, mask_test, delta_test, static_test,
            y_test_soft, n_epochs=100, learning_rate=0.01
        )

       # Evaluate fine-tuned model on test set
        fine_tuned_model.eval()
        with torch.no_grad():
            probs = fine_tuned_model(x_test, mask_test, delta_test, static_test).squeeze().cpu().numpy()
        
        def evaluate_predictions(y_true, y_prob, threshold=threshold):
            y_pred = (y_prob > threshold).astype(int)
            return {
                "Outcome": outcome,
                "Precision": precision_score(y_true, y_pred, zero_division=0).round(3),
                "Recall": recall_score(y_true, y_pred, zero_division=0).round(3),
                "F1 Score": f1_score(y_true, y_pred, zero_division=0).round(3),
                "ROC-AUC": roc_auc_score(y_true, y_prob).round(3),
                "threshold": threshold.round(3)
            }
        
        # Evaluate and print results
        metrics = evaluate_predictions(y_test_true, probs, threshold)
        
        return metrics


In [2]:
outcomes = ["STILLBIRTH_SIGNS_LIFE","LBW2500_ANY","INF_PSBI_IPC","SVN", "NEARMISS", "NEO_DTH"]
probs = ['stillbirth.prob', 'lbw.prob', 'psbi.prob', 'svn.prob', 'nearmiss.prob','neo_dth.prob']

# Function to process results for an outcome
def process_results(outcome,prob):
    # Run the function and capture the returned values
    results_summary= print_results(outcome,prob)
    print(results_summary)
    
# Run the function for each outcome
for outcome, prob in zip(outcomes, probs):
    process_results(outcome,prob)

{'Outcome': 'STILLBIRTH_SIGNS_LIFE', 'Precision': 0.208, 'Recall': 0.763, 'F1 Score': 0.326, 'ROC-AUC': 0.927, 'threshold': 0.027}
{'Outcome': 'LBW2500_ANY', 'Precision': 0.31, 'Recall': 0.587, 'F1 Score': 0.406, 'ROC-AUC': 0.749, 'threshold': 0.151}
{'Outcome': 'INF_PSBI_IPC', 'Precision': 0.092, 'Recall': 0.571, 'F1 Score': 0.158, 'ROC-AUC': 0.651, 'threshold': 0.061}
{'Outcome': 'SVN', 'Precision': 0.261, 'Recall': 0.442, 'F1 Score': 0.328, 'ROC-AUC': 0.617, 'threshold': 0.18}
{'Outcome': 'NEARMISS', 'Precision': 0.249, 'Recall': 0.584, 'F1 Score': 0.349, 'ROC-AUC': 0.776, 'threshold': 0.084}
{'Outcome': 'NEO_DTH', 'Precision': 0.051, 'Recall': 0.87, 'F1 Score': 0.097, 'ROC-AUC': 0.923, 'threshold': 0.009}
